In [1]:
!pip install flask flask-sqlalchemy


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\DEEVONIKA HOOJA\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [20]:
import tkinter as tk
from tkinter import messagebox
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

In [21]:
# Initialize Flask app
flask_app = Flask(__name__)  # Corrected from _name_ to __name__
flask_app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///finansavvy.db'
flask_app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

In [22]:
# Initialize database
db = SQLAlchemy(flask_app)

In [23]:
# Models
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    password = db.Column(db.String(255), nullable=False)
    transactions = db.relationship('Transaction', backref='user', lazy=True)
    budgets = db.relationship('Budget', backref='user', lazy=True)
    goals = db.relationship('Goal', backref='user', lazy=True)

class Transaction(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    amount = db.Column(db.Float, nullable=False)
    description = db.Column(db.String(255), nullable=False)
    category = db.Column(db.String(100), nullable=False)
    date = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

class Budget(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    category = db.Column(db.String(100), nullable=False)
    amount = db.Column(db.Float, nullable=False)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

class Goal(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    target_amount = db.Column(db.Float, nullable=False)
    current_amount = db.Column(db.Float, nullable=False, default=0)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)


In [24]:
# Tkinter GUI
class FinanceApp:
    def __init__(self, root):  # Corrected from _init_ to __init__
        self.root = root
        self.root.title("Financial Management System")
        self.user = None  # To track the logged-in user

        self.show_login_page()

    def show_login_page(self):
        self.clear_window()

        self.username_label = tk.Label(self.root, text="Username:")
        self.username_label.grid(row=0, column=0, padx=10, pady=10)
        self.username_entry = tk.Entry(self.root)
        self.username_entry.grid(row=0, column=1, padx=10, pady=10)

        self.password_label = tk.Label(self.root, text="Password:")
        self.password_label.grid(row=1, column=0, padx=10, pady=10)
        self.password_entry = tk.Entry(self.root, show="*")
        self.password_entry.grid(row=1, column=1, padx=10, pady=10)

        self.login_button = tk.Button(self.root, text="Login", command=self.login_user)
        self.login_button.grid(row=2, column=0, padx=10, pady=10)

        self.signup_button = tk.Button(self.root, text="Sign Up", command=self.signup_user)
        self.signup_button.grid(row=2, column=1, padx=10, pady=10)

    def clear_window(self):
        for widget in self.root.winfo_children():
            widget.destroy()

    def login_user(self):
        username = self.username_entry.get()
        password = self.password_entry.get()

        if username and password:
            with flask_app.app_context():
                user = User.query.filter_by(username=username, password=password).first()
                if user:
                    self.user = user  # Set the logged-in user
                    messagebox.showinfo("Success", "Login successful!")
                    self.show_main_page()  # Show main page after login
                else:
                    messagebox.showerror("Error", "Invalid username or password.")
        else:
            messagebox.showwarning("Warning", "Please fill all fields.")

    def signup_user(self):
        username = self.username_entry.get()
        password = self.password_entry.get()

        if username and password:
            with flask_app.app_context():
                user_exists = User.query.filter_by(username=username).first()
                if user_exists:
                    messagebox.showinfo("Info", "Username already exists.")
                else:
                    new_user = User(username=username, email=f"{username}@example.com", password=password)
                    db.session.add(new_user)
                    db.session.commit()
                    messagebox.showinfo("Success", "Sign up successful! You can now log in.")
        else:
            messagebox.showwarning("Warning", "Please fill all fields.")

    def show_main_page(self):
        self.clear_window()

        # Transaction section
        self.transaction_amount_label = tk.Label(self.root, text="Transaction Amount:")
        self.transaction_amount_label.grid(row=0, column=0, padx=10, pady=10)
        self.transaction_amount_entry = tk.Entry(self.root)
        self.transaction_amount_entry.grid(row=0, column=1, padx=10, pady=10)

        self.transaction_desc_label = tk.Label(self.root, text="Description:")
        self.transaction_desc_label.grid(row=1, column=0, padx=10, pady=10)
        self.transaction_desc_entry = tk.Entry(self.root)
        self.transaction_desc_entry.grid(row=1, column=1, padx=10, pady=10)

        self.transaction_category_label = tk.Label(self.root, text="Category:")
        self.transaction_category_label.grid(row=2, column=0, padx=10, pady=10)
        self.transaction_category_entry = tk.Entry(self.root)
        self.transaction_category_entry.grid(row=2, column=1, padx=10, pady=10)

        self.add_transaction_button = tk.Button(self.root, text="Add Transaction", command=self.add_transaction)
        self.add_transaction_button.grid(row=3, column=1, padx=10, pady=10)

        # Budget section
        self.budget_category_label = tk.Label(self.root, text="Budget Category:")
        self.budget_category_label.grid(row=4, column=0, padx=10, pady=10)
        self.budget_category_entry = tk.Entry(self.root)
        self.budget_category_entry.grid(row=4, column=1, padx=10, pady=10)

        self.budget_amount_label = tk.Label(self.root, text="Budget Amount:")
        self.budget_amount_label.grid(row=5, column=0, padx=10, pady=10)
        self.budget_amount_entry = tk.Entry(self.root)
        self.budget_amount_entry.grid(row=5, column=1, padx=10, pady=10)

        self.add_budget_button = tk.Button(self.root, text="Add Budget", command=self.add_budget)
        self.add_budget_button.grid(row=6, column=1, padx=10, pady=10)

        # Goal section
        self.goal_name_label = tk.Label(self.root, text="Goal Name:")
        self.goal_name_label.grid(row=7, column=0, padx=10, pady=10)
        self.goal_name_entry = tk.Entry(self.root)
        self.goal_name_entry.grid(row=7, column=1, padx=10, pady=10)

        self.goal_target_label = tk.Label(self.root, text="Target Amount:")
        self.goal_target_label.grid(row=8, column=0, padx=10, pady=10)
        self.goal_target_entry = tk.Entry(self.root)
        self.goal_target_entry.grid(row=8, column=1, padx=10, pady=10)

        self.goal_current_label = tk.Label(self.root, text="Current Amount:")
        self.goal_current_label.grid(row=9, column=0, padx=10, pady=10)
        self.goal_current_entry = tk.Entry(self.root)
        self.goal_current_entry.grid(row=9, column=1, padx=10, pady=10)

        self.add_goal_button = tk.Button(self.root, text="Add Goal", command=self.add_goal)
        self.add_goal_button.grid(row=10, column=1, padx=10, pady=10)

    def add_transaction(self):
        amount = self.transaction_amount_entry.get()
        description = self.transaction_desc_entry.get()
        category = self.transaction_category_entry.get()

        if amount and description and category:
            with flask_app.app_context():
                new_transaction = Transaction(
                    amount=float(amount),
                    description=description,
                    category=category,
                    user_id=self.user.id
                )
                db.session.add(new_transaction)
                db.session.commit()
                messagebox.showinfo("Success", "Transaction added successfully!")
        else:
            messagebox.showwarning("Warning", "Please fill all fields.")

    def add_budget(self):
        category = self.budget_category_entry.get()
        amount = self.budget_amount_entry.get()

        # Check if 'amount' is a valid float
        try:
            amount = float(amount)
        except ValueError:
            messagebox.showerror("Error", "Please enter a valid number for the budget amount.")
            return

        if category and amount:
            with flask_app.app_context():
                new_budget = Budget(
                    category=category,
                    amount=amount,
                    user_id=self.user.id
                )
                db.session.add(new_budget)
                db.session.commit()
                messagebox.showinfo("Success", "Budget added successfully!")
        else:
            messagebox.showwarning("Warning", "Please fill all fields.")

    def add_goal(self):
        name = self.goal_name_entry.get()
        target_amount = self.goal_target_entry.get()
        current_amount = self.goal_current_entry.get()

        if name and target_amount and current_amount:
            with flask_app.app_context():
                new_goal = Goal(
                    name=name,
                    target_amount=float(target_amount),
                    current_amount=float(current_amount),
                    user_id=self.user.id
                )
                db.session.add(new_goal)
                db.session.commit()
                messagebox.showinfo("Success", "Goal added successfully!")
        else:
            messagebox.showwarning("Warning", "Please fill all fields.")


In [25]:
# Initialize the Tkinter loop
if __name__ == "__main__":
    with flask_app.app_context():
        db.create_all()

    root = tk.Tk()
    app = FinanceApp(root)
    root.mainloop()